In [22]:
from gurobipy import *
import numpy as np
import pandas as pd

# Structure


In [23]:
Biomass  = pd.read_csv("pred_2019.csv")["Min1819"].values

In [24]:
Biomass.sum()

344599.9615984451

In [25]:
UB = pd.read_csv("pred_2019.csv")["2018"].values

In [26]:
UB.sum() * 0.8

284792.43719617085

In [27]:
DistanceB = pd.read_csv("Distance_Matrix.csv").drop(columns = ["INDEX"])

In [28]:
DistanceB.head()

,0,1,2,3,4,5,6,7,8,9,...,2408,2409,2410,2411,2412,2413,2414,2415,2416,2417
0,0.0000,11.3769,20.4557,38.1227,45.3810,54.9915,78.6108,118.6750,102.6639,113.4309,...,683.8771,687.6310,697.3246,669.3962,667.6788,665.5775,662.0291,665.9655,673.2073,681.4235
1,11.3769,0.0000,9.0788,28.9141,36.1724,45.7829,69.4022,78.2329,93.4553,111.1832,...,681.6295,685.3833,695.0769,667.1485,665.4311,663.3298,659.7815,663.7178,670.9596,679.1758
2,20.4557,9.0788,0.0000,22.3791,29.6374,39.2478,62.8671,71.6979,86.9203,111.7859,...,682.2323,685.9861,695.6796,667.7513,666.0339,663.9326,660.3843,664.3206,671.5623,679.7786
3,38.1227,28.9141,22.3791,0.0000,11.8343,23.5413,41.8396,50.6703,65.8927,82.5852,...,681.4226,685.1765,694.8701,666.9417,665.2243,663.1230,659.5746,663.5110,670.7528,678.9690
4,45.3810,36.1724,29.6374,11.8343,0.0000,11.7070,24.3986,33.2293,53.9901,65.1442,...,663.9816,667.7355,677.4291,649.5007,647.7833,645.6820,642.1336,646.0700,653.3118,661.5280


In [29]:
DistanceB = DistanceB.values
DistanceB.shape

(2418, 2418)

In [30]:
DistanceP = DistanceB.copy()

In [31]:
Di,Dj = DistanceB.shape 

In [32]:
M = 10**9
for i in range(Di):
    for j in range(Dj):
        if(i == j):
            DistanceB[i][j] = M
            DistanceP[i][j] = 0

In [33]:
print(DistanceB)

[[1.000000e+09 1.137690e+01 2.045570e+01 ... 6.659655e+02 6.732073e+02
  6.814235e+02]
 [1.137690e+01 1.000000e+09 9.078800e+00 ... 6.637178e+02 6.709596e+02
  6.791758e+02]
 [2.045570e+01 9.078800e+00 1.000000e+09 ... 6.643206e+02 6.715623e+02
  6.797786e+02]
 ...
 [6.637748e+02 6.615271e+02 6.621298e+02 ... 1.000000e+09 1.424230e+01
  2.245860e+01]
 [6.710165e+02 6.687688e+02 6.693715e+02 ... 1.424230e+01 1.000000e+09
  1.247410e+01]
 [6.792328e+02 6.769851e+02 6.775878e+02 ... 2.245860e+01 1.247410e+01
  1.000000e+09]]


In [34]:
print(DistanceP)

[[  0.      11.3769  20.4557 ... 665.9655 673.2073 681.4235]
 [ 11.3769   0.       9.0788 ... 663.7178 670.9596 679.1758]
 [ 20.4557   9.0788   0.     ... 664.3206 671.5623 679.7786]
 ...
 [663.7748 661.5271 662.1298 ...   0.      14.2423  22.4586]
 [671.0165 668.7688 669.3715 ...  14.2423   0.      12.4741]
 [679.2328 676.9851 677.5878 ...  22.4586  12.4741   0.    ]]


# Variables


In [21]:
m = Model("Shell.ai.try")

In [13]:
B = m.addMVar((Di,Dj), vtype= GRB.CONTINUOUS , lb = 0 , name = "B" )

In [14]:
P = m.addMVar((Di,Dj), vtype= GRB.CONTINUOUS , lb = 0 , name= "P")

In [15]:
X = m.addMVar((Di), vtype= GRB.BINARY, name = "X")
Y = m.addMVar((Di), vtype= GRB.BINARY , name = "Y")
Z = m.addMVar((Di), vtype= GRB.BINARY, name = "Z")

In [16]:
Xm = m.addMVar((Di) , vtype= GRB.CONTINUOUS, name = "Xm", lb = 0)
Ym = m.addMVar((Di) , vtype= GRB.CONTINUOUS, name = "Ym", lb = 0)
Zm = m.addMVar((Di) , vtype= GRB.CONTINUOUS, name = "Zm" ,lb = 0)

In [ ]:
babs = m.addVar(vtype= GRB.CONTINUOUS , name = "babs" , lb = 0)

# Objective

In [18]:
import gurobipy as gp

In [19]:
exp1 = (DistanceB*B).sum()

In [20]:
exp1 += (DistanceP*P).sum()

In [21]:
exp2 = (20000*Y - B.sum(axis=0)).sum()

In [22]:
exp3 = (100000*Z - P.sum(axis=0)).sum()

In [23]:
m.setObjective(exp1 + exp2 + exp3 , GRB.MINIMIZE)

In [24]:
m.update()

# Constraints

In [25]:
x_mask = []
for i in range(Di):
    x_mask = B[i].tolist()
    m.addConstr(Xm[i] == max_(x_mask,constant = 0), name=f"C1m{i}")
    m.addConstr(X[i] == min_(Xm[i],constant = 1), name=f"C1{i}")

In [26]:
y_mask = []
for i in range(Dj):
    y_mask = B[:,i].tolist()
    m.addConstr(Ym[i] == max_(y_mask,constant = 0), name=f"C2m{i}")
    m.addConstr(Y[i] == min_(Ym[i],constant = 1), name=f"C2{i}")

In [27]:
z_mask = []
for i in range(Dj):
    z_mask = P[:,i].tolist()
    m.addConstr(Zm[i] == max_(z_mask,constant = 0), name=f"C3m{i}")
    m.addConstr(Z[i] == min_(Zm[i],constant = 1), name=f"C3{i}")

In [ ]:
z_mask = []
for i in range(Dj):
    z_mask = P[i].tolist()
    m.addConstr(Zm[i] == max_(z_mask,constant = 0), name=f"C32_m{i}")
    m.addConstr(Y[i] == min_(Ym[i],constant = 1), name=f"C32_{i}")

In [ ]:
B_sum = B.sum(axis=0)
P_sum = P.sum(axis=1)

In [ ]:
for i in range(Di):
    m.addConstr(B_sum[i] == P_sum[i])

In [28]:
B_sum = B.sum(axis=1)
for i in range(Di):
    m.addConstr(B_sum[i] - Biomass[i] <= 0 , name=f"C_4_{i}")

In [29]:
B_sum = B.sum(axis=0)
for j in range(Dj):
    m.addConstr(B_sum[j] - 20000 <= 0 , name = f"C_5_{j}")

In [30]:
P_sum = P.sum(axis =0)
for j in range(Dj):
    m.addConstr(P_sum[j] - 100000 <= 0 , name = f"C_6_{j}")

In [31]:
con7 = (Y.sum() <= 25)

In [32]:
m.addConstr(con7 , name = "C_7")

<MConstr () *awaiting model update*>

In [33]:
con8 = (Z.sum() <= 5)

In [34]:
m.addConstr(con8 , name = "C_8")

<MConstr () *awaiting model update*>

In [35]:
con9 = (B.sum() >= ((0.8*(UB.sum())+ 1))

In [36]:
m.addConstr(con9 , name = "C_9")

<MConstr () *awaiting model update*>

In [ ]:
m.addConstr(babs == (B.sum() - P.sum()) , name = "C_10_1")

In [ ]:
m.addConstr(babs <= 0.0001, name = "C_10_2")

In [38]:
for i in range(Di):
    m.addConstr(X[i] + Y[i] <= 1, name = f"C_11_{i}" )

In [39]:
for i in range(Di):
    m.addConstr(X[i] + Z[i] <= 1, name = f'C_12_{i}')

In [41]:
m.update()

In [42]:
m.write("1819.lp")

In [74]:
# m.setParam("NoRelHeurTime",80)
# m.setParam("MIPGap",0.4)
# m.setParam("MIPFocus",1)

In [75]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-12450H, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 505 rows, 20602 columns and 70602 nonzeros
Model fingerprint: 0xd4e8a246
Model has 600 general constraints
Variable types: 20302 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+09]
  Bounds range     [3e-03, 1e+00]
  RHS range        [1e+00, 4e+04]
  GenCon const rng [1e+00, 1e+00]
Presolved: 61172 rows, 80569 columns, 379458 nonzeros

Continuing optimization...


Cutting planes:
  Gomory: 5
  Cover: 138
  Implied bound: 213
  MIR: 26
  Flow cover: 327
  Zero half: 2
  Relax-and-lift: 80
  BQP: 6

Explored 1 nodes (27722 simplex iterations) in 0.09 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 4.47656e+06 

Optimal solution fou

In [65]:
obj = m.getObjective()

In [66]:
print(obj.getValue())

4476563.200000001


In [47]:
m.write("1819.sol")
m.write('1819.mps')

# Extract results

In [79]:
def Sol_pipeline(m):
    
    # Biomass
    
    res = []
    for i in range(Di*Dj):
        res.append(m.getVars()[i].X)
    B_res = np.array(res)
    B_res = B_res.reshape(Di,Dj)
    idx = Di*Dj 
    
    # Pellets
    
    res.clear()
    for i in range(idx,(2*idx)):
        res.append(m.getVars()[i].X)
    P_res = np.array(res).reshape(Di,Dj)
    idx = 2*idx
    
    # Harvesting sites
    
    res.clear()
    for i in range(idx,(idx+Di)):
        res.append(m.getVars()[i].X)
    X_res = np.array(res)
    idx = idx + Di
    
    # Depot sites
    
    res.clear()
    for i in range(idx,idx+Di):
        res.append(m.getVars()[i].X)
    Y_res = np.array(res)
    idx = idx + Di
    
    # Refinery sites
    
    res.clear()
    for i in range(idx,idx+Di):
        res.append(m.getVars()[i].X)
    Z_res = np.array(res)
    
    return {"B" : B_res , "P" : P_res , "X" : X_res , "Y" : Y_res , "Z" : Z_res}

In [80]:
result = Sol_pipeline(m)

In [81]:
B_res = result["B"]
P_res = result["P"]
X_res = result["X"]
Y_res = result["Y"]
Z_res = result["Z"]

In [ ]:
P_sum = np.zeros((Dj))
B_sum = np.zeros((Di))
Depot_sum = np.zeros((Di))
Refinery_sum = np.zeros((Dj))

In [ ]:
# Round off
# for i in range(Di):
#     for j in range(Dj):
#         B_res[i][j] = round(B_res[i][j],4)
B_sum = B_res.sum(axis = 1)
P_sum = P_res.sum(axis = 1)
Depot_sum = B_res.sum(axis = 0)
Refinery_sum = P_res.sum(axis = 0)

In [ ]:
B_res.sum()

# Checks

In [83]:
# All values (forecasted biomass, biomass demand-supply, pellet demand-supply) must be
# greater than or equal to zero. 

print(False not in (Biomass >= 0),
False not in (B_res >= 0),
False not in (P_res >= 0 ))

False

In [84]:
# The amount of biomass procured for processing from each harvesting site ′𝑖′ must be less than
# or equal to that site’s forecasted biomass.

False not in (Biomass  - B_sum >= 0 )

3.0

In [85]:
# Total biomass reaching each preprocessing depot ′𝑗′ must be less than or equal to its yearly
# processing capacity (20,000).

False not in (Depot_sum <= 20000)

False

In [86]:
# Total pellets reaching each refinery ′𝑘′ must be less than or equal to its yearly processing
# capacity (100,000).

False not in (Refinery_sum <= 100000)

False

In [87]:
# Number of depots should be less than or equal to 25.

(Y_res.sum() <= 25)

32475.199999999997

In [88]:
# Number of refineries should be less than or equal to 5.

(Z_res.sum() <= 5)

16475.199999999997

In [89]:
# At least 80% of the total forecasted biomass must be processed by refineries each year.
(B_res.sum() >= (0.8 * UB.sum()))

4427612.8

In [90]:
# Total amount of biomass entering each preprocessing depot is equal to the total amount of
# pellets exiting that depot (within tolerance limit of 1e-03).

(B_res.sum() - P_res.sum()) <= 0.001 

4083874.999369733

# Objective estimates

In [92]:
residual1 = (Y_res.sum()*4000 - B_sum.sum())

True

In [93]:
residual2 = (Z_res.sum()*20000 - P_sum.sum())

43524.8

In [94]:
transcost = (DistanceB*B_res).sum() + (DistanceP*P_res).sum()

43524.8

In [104]:
estobj = residual1 + residual2 + transcost

In [105]:
estobj - obj.getValue()

# Final output

In [ ]:
pd.DataFrame(Y_res).to_csv("Depots.csv",index = False, columns = ["Loc"])
pd.DataFrame(Z_res).to_csv("Refineries.csv",index = False , columns = ["Loc"])